### Libgen bulk downloader
- https://github.com/harrison-broadbent/libgen-api

Este es un programa para automatizar la busqueda y descarga de libros de relevancia,
a partir de las sugerencias dadas por AI y obtencion de contenidos en Libgen.
El codigo contiene 3 partes:
1. Consulta a AI: conexion a Bard API y extraccion de respuesta como diccionario de python
2. Obtencion de links: conexion a Libgen API para obtencion de links de descarga
3. Descarga de archivos: uso de terminal para nombrar archivos y consolidar resultados en una carpeta

INPUTS DE USUARIO

1. limite de libros
2. consulta a Bard

OUTPUT

El resultado final es una carpeta en el directorio local con los contenidos descargados

TODO LIST

El resultado final es un directorio cuyo nombre es el tema de busqueda, y el contenido sean los archivos descargados
1. Usar bard api para generar una consulta 
2. Limpiar la respuesta para generar un diccionario de recomendaciones
3. Usar como input el punto 2 para realizar las busquedas de metadata en libgen
4. Usar la metadata del punto 3 para generar los links de descarga
5. Descargar a traves de los links usando la terminal
6. guardar resultados en una carpeta local
    - armar dos subcarpetas: una para pdf y otra para epub o mobi

ADICIONALES
7. Usar la libreria click para armar todo como un CLI
8. Agregar holistic approach para buscar tanto autor como titulo de libro


In [16]:
import bardapi
from libgen_api import LibgenSearch
import subprocess
import re
import libgen_api
print(libgen_api.__file__)
from bardapi import Bard

c:\Users\juanm\miniconda3\Lib\site-packages\libgen_api\__init__.py


### BARD

In [21]:
# USER INPUTS
topic = 'Macroeconomics'
limit = 5

In [20]:
bard_token = 'aAju5-oGJDHaMYWKImrm9Wef-IV4bs0deuWjfMag2PcornrmNGttVNyB1kkVNk9bVBqhXA.'

In [ ]:
#bard = Bard(token_from_browser=True)
#res = bard.get_answer("Do you like cookies?")
#print(res)

In [22]:
input_text = f"Create a list of the best {limit} books related to {topic}. Return your response as a python dictionary, with the author name as the key and book title as a value. Your answer has to contain ONLY the python dictionary!"
#bard_response = bard.get_answer(input_text)['content']
response = bardapi.core.Bard(bard_token).get_answer(input_text)

In [23]:
response['code']

'\n{\n    "Mankiw": "Principles of Economics",\n    "Romer": "Advanced Macroeconomics",\n    "Krugman": "Macroeconomics",\n    "Blanchard": "Macroeconomics",\n    "Stiglitz": "The Roaring Nineties"\n}\n'

In [24]:
# Extract dict from response string
pattern = r"\{(.*?)\}"
match = re.search(pattern, response['code'], re.DOTALL)
if match:
    extracted_content = match.group(1)
    final_str = "{" + str(extracted_content) + "}"
    books_dict = eval(final_str)
    print(books_dict)

{'Mankiw': 'Principles of Economics', 'Romer': 'Advanced Macroeconomics', 'Krugman': 'Macroeconomics', 'Blanchard': 'Macroeconomics', 'Stiglitz': 'The Roaring Nineties'}


### LIBGEN

In [25]:
dummy_data = {'Mankiw': 'Principles of Economics', 'Romer': 'Advanced Macroeconomics', 'Krugman': 'Macroeconomics', 'Blanchard': 'Macroeconomics', 'Stiglitz': 'The Roaring Nineties'}

In [3]:
col_names = [
        "ID",
        "Author",
        "Title",
        "Publisher",
        "Year",
        "Pages",
        "Language",
        "Size",
        "Extension",
        "Mirror_1",
        "Mirror_2",
        "Mirror_3",
        "Mirror_4",
        "Mirror_5",
        "Edit",
    ]

In [26]:
libgen = LibgenSearch()

In [30]:
for k,v in books_dict.items():
    search_prompt = k + ' ' +  v
    print(search_prompt)

Mankiw Principles of Economics
Romer Advanced Macroeconomics
Krugman Macroeconomics
Blanchard Macroeconomics
Stiglitz The Roaring Nineties


In [31]:
filters = {"Language": "English", "Extension": "pdf"}
metadata_list = []
for k,v in books_dict.items():
    search_prompt = k + ' ' +  v
    book_search = libgen.search_holistic_filtered(f"{search_prompt}", filters, exact_match=False)
    metadata_list.append(book_search)
    print(book_search)

[{'ID': '26907', 'Author': 'Mankiw N.G.', 'Title': 'Ten principles of economics', 'Publisher': '', 'Year': '2004', 'Pages': '508', 'Language': 'English', 'Size': '4 Mb', 'Extension': 'pdf', 'Mirror_1': 'http://library.lol/main/3C867F5BBE7352C53DDA722D138D1820', 'Mirror_2': 'http://libgen.lc/ads.php?md5=3C867F5BBE7352C53DDA722D138D1820', 'Mirror_3': 'https://library.bz/main/edit/3C867F5BBE7352C53DDA722D138D1820'}, {'ID': '79071', 'Author': 'Mankiw', 'Title': 'Principles of Economics HQ', 'Publisher': '', 'Year': '', 'Pages': '793', 'Language': 'English', 'Size': '6 Mb', 'Extension': 'pdf', 'Mirror_1': 'http://library.lol/main/91557FF34BEE7291415C884C6F1E75A0', 'Mirror_2': 'http://libgen.lc/ads.php?md5=91557FF34BEE7291415C884C6F1E75A0', 'Mirror_3': 'https://library.bz/main/edit/91557FF34BEE7291415C884C6F1E75A0'}, {'ID': '79074', 'Author': 'Mankiw', 'Title': 'Principles of Economics', 'Publisher': '', 'Year': '2003', 'Pages': '791', 'Language': 'English', 'Size': '6 Mb', 'Extension': 'pdf

In [80]:
title_filters = {"Language": "English", "Extension": "pdf"}
titles = libgen.search_title_filtered("Macroeconomics", title_filters, exact_match=True)
titles

#books_dict.values()

[{'ID': '223089',
  'Author': 'N. Gregory Mankiw',
  'Title': 'Principles of Macroeconomics (Canadian Edition)',
  'Publisher': 'South-Western Educational Publishing',
  'Year': '2002',
  'Pages': '181',
  'Language': 'English',
  'Size': '22 Mb',
  'Extension': 'pdf',
  'Mirror_1': 'http://library.lol/main/B7EE3D47B897CFBE6C90872F0FC57874',
  'Mirror_2': 'http://libgen.lc/ads.php?md5=B7EE3D47B897CFBE6C90872F0FC57874',
  'Mirror_3': 'https://library.bz/main/edit/B7EE3D47B897CFBE6C90872F0FC57874'},
 {'ID': '12623',
  'Author': 'Farmer R.E.A.',
  'Title': 'Macroeconomics',
  'Publisher': '',
  'Year': '1997',
  'Pages': '372',
  'Language': 'English',
  'Size': '3 Mb',
  'Extension': 'pdf',
  'Mirror_1': 'http://library.lol/main/7C4A0DFC27B30E3BBA4B84D736C3B37E',
  'Mirror_2': 'http://libgen.lc/ads.php?md5=7C4A0DFC27B30E3BBA4B84D736C3B37E',
  'Mirror_3': 'https://library.bz/main/edit/7C4A0DFC27B30E3BBA4B84D736C3B37E'},
 {'ID': '12657',
  'Author': 'N. Gregory(N. Gregory Mankiw) Mankiw',


In [8]:
results = libgen.search_author("Jane Austen")
download_all_links = dict()
for i in range(len(results)):
    item_to_download = results[i]
    download_links = libgen.resolve_download_links(item_to_download)
    

print(download_links)

[{'ID': '261367', 'Author': 'Jane Austen,Margaret Drabble', 'Title': 'Persuasion', 'Publisher': 'Signet Classics', 'Year': '1996', 'Pages': '221', 'Language': 'English', 'Size': '520 Kb', 'Extension': 'pdf', 'Mirror_1': 'http://library.lol/main/9A69D873CB6C3A1FB071F00C1736BEA5', 'Mirror_2': 'http://libgen.lc/ads.php?md5=9A69D873CB6C3A1FB071F00C1736BEA5', 'Mirror_3': 'https://library.bz/main/edit/9A69D873CB6C3A1FB071F00C1736BEA5'}, {'ID': '261368', 'Author': 'Jane Austen,R. W. Chapman', 'Title': 'Lady Susan', 'Publisher': 'Dover Publications', 'Year': '2005', 'Pages': '71', 'Language': 'English', 'Size': '254 Kb', 'Extension': 'pdf', 'Mirror_1': 'http://library.lol/main/E891648418CC8E1CCF31DE664CF14FC2', 'Mirror_2': 'http://libgen.lc/ads.php?md5=E891648418CC8E1CCF31DE664CF14FC2', 'Mirror_3': 'https://library.bz/main/edit/E891648418CC8E1CCF31DE664CF14FC2'}, {'ID': '261779', 'Author': 'Jane Austen,Claudia L. Johnson', 'Title': 'Mansfield Park', 'Publisher': 'W. W. Norton & Company', 'Year

In [16]:
len(results)

25

[{'ID': '402153',
  'Author': 'Agatha Christie',
  'Title': 'Murder on the Orient Express: A Hercule Poirot Mystery',
  'Publisher': '',
  'Year': '1934',
  'Pages': '',
  'Language': 'English',
  'Size': '197 Kb',
  'Extension': 'epub',
  'Mirror_1': 'http://library.lol/main/84959F8604B07C439A9473A6E645668A',
  'Mirror_2': 'http://libgen.lc/ads.php?md5=84959F8604B07C439A9473A6E645668A',
  'Mirror_3': 'https://library.bz/main/edit/84959F8604B07C439A9473A6E645668A'},
 {'ID': '411350',
  'Author': 'Agatha Christie',
  'Title': 'The Mystery of the Blue Train',
  'Publisher': '',
  'Year': '',
  'Pages': '',
  'Language': 'English',
  'Size': '239 Kb',
  'Extension': 'epub',
  'Mirror_1': 'http://library.lol/main/79E6C529F6962F81AC46EB66FBECE715',
  'Mirror_2': 'http://libgen.lc/ads.php?md5=79E6C529F6962F81AC46EB66FBECE715',
  'Mirror_3': 'https://library.bz/main/edit/79E6C529F6962F81AC46EB66FBECE715'}]

In [ ]:
# Correr proceso en terminal para realizar las descargas
p = subprocess.run()